In [9]:
import requests
import csv
import time

# Define the URL for the streaming endpoint
url = 'https://www.floods.it/api/v1/monitoring/38561.csv'

# Open a CSV file to write the data
with open('sensor_data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header row
    writer.writerow(['IdSensore', 'Data/Ora', 'Valore', 'Validato','OraSolare'])

    # Connect to the URL and stream data
    response = requests.get(url, stream=True)

    if response.status_code == 200:
        print("Connected to the stream. Writing data to CSV...")
        for line in response.iter_lines():
            if line:
                # Decode the byte stream and process the data
                data = line.decode('utf-8')
                # Assume data is in JSON format, parse it accordingly
                try:
                    import json
                    json_data = json.loads(data)
                    # Extract relevant fields (this will depend on your data format)
                    IdSensor = json_data.get('IdSensore')
                    timestamp = json_data.get('Data/Ora')
                    value = json_data.get('Valore')
                    valstate = json_data.get('Validato')
                    solhour = json_data.get('OraSolare')
                    # Write the data to the CSV file
                    writer.writerow([timestamp, IdSensor, value])
                    print(f"Wrote data: {timestamp}, {IdSensor}, {value}, {valstate}, {solhour}")
                except json.JSONDecodeError:
                    print("Failed to decode JSON:", data)
    else:
        print(f"Failed to connect to {url}. Status code: {response.status_code}")


Connected to the stream. Writing data to CSV...
Failed to decode JSON: time_format,%Y-%m-%d %H:%M:%S
Failed to decode JSON: tz,UTC+01
Failed to decode JSON: api_warnings,
Failed to decode JSON: timestamp,value
Failed to decode JSON: 2024-05-14 00:00:00,5.822
Failed to decode JSON: 2024-05-14 00:15:00,5.633
Failed to decode JSON: 2024-05-14 00:30:00,5.784
Failed to decode JSON: 2024-05-14 00:45:00,5.746
Failed to decode JSON: 2024-05-14 01:00:00,5.596
Failed to decode JSON: 2024-05-14 01:15:00,5.708
Failed to decode JSON: 2024-05-14 01:30:00,5.596
Failed to decode JSON: 2024-05-14 01:45:00,5.746
Failed to decode JSON: 2024-05-14 02:00:00,5.708
Failed to decode JSON: 2024-05-14 02:15:00,5.746
Failed to decode JSON: 2024-05-14 02:30:00,5.671
Failed to decode JSON: 2024-05-14 02:45:00,5.784
Failed to decode JSON: 2024-05-14 03:00:00,5.746
Failed to decode JSON: 2024-05-14 03:15:00,5.558
Failed to decode JSON: 2024-05-14 03:30:00,5.708
Failed to decode JSON: 2024-05-14 03:45:00,5.671
Failed

In [8]:
import requests
import csv
import time

# Define the URL of the endpoint
url = "https://www.floods.it/api/v1/monitoring/38561.csv"

# Define the CSV file where data will be stored
output_csv_file = "iot_sensor_data.csv"

def stream_to_csv(url, output_csv_file):
    # Open the CSV file in append mode
    with open(output_csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        
        # Connect to the URL
        response = requests.get(url, stream=True)
        
        # Check if the connection was successful
        if response.status_code == 200:
            print("Connection established. Streaming data...")
            try:
                for line in response.iter_lines():
                    if line:
                        # Decode the line and split by comma
                        decoded_line = line.decode('utf-8')
                        data = decoded_line.split(',')
                        
                        # Write the data to the CSV file
                        writer.writerow(data)
                        print(f"Data written to CSV: {data}")
            except KeyboardInterrupt:
                print("Stream interrupted by user.")
            except Exception as e:
                print(f"An error occurred: {e}")
            finally:
                print("Closing the connection.")
        else:
            print(f"Failed to connect to {url}. Status code: {response.status_code}")

if __name__ == "__main__":
    while True:
        try:
            stream_to_csv(url, output_csv_file)
        except Exception as e:
            print(f"An error occurred: {e}")
        time.sleep(5)  # Wait for a few seconds before attempting to reconnect


Connection established. Streaming data...
Data written to CSV: ['time_format', '%Y-%m-%d %H:%M:%S']
Data written to CSV: ['tz', 'UTC+01']
Data written to CSV: ['api_warnings', '']
Data written to CSV: ['timestamp', 'value']
Data written to CSV: ['2024-05-14 00:00:00', '5.822']
Data written to CSV: ['2024-05-14 00:15:00', '5.633']
Data written to CSV: ['2024-05-14 00:30:00', '5.784']
Data written to CSV: ['2024-05-14 00:45:00', '5.746']
Data written to CSV: ['2024-05-14 01:00:00', '5.596']
Data written to CSV: ['2024-05-14 01:15:00', '5.708']
Data written to CSV: ['2024-05-14 01:30:00', '5.596']
Data written to CSV: ['2024-05-14 01:45:00', '5.746']
Data written to CSV: ['2024-05-14 02:00:00', '5.708']
Data written to CSV: ['2024-05-14 02:15:00', '5.746']
Data written to CSV: ['2024-05-14 02:30:00', '5.671']
Data written to CSV: ['2024-05-14 02:45:00', '5.784']
Data written to CSV: ['2024-05-14 03:00:00', '5.746']
Data written to CSV: ['2024-05-14 03:15:00', '5.558']
Data written to CSV:

KeyboardInterrupt: 

In [13]:
import requests
import pandas as pd
from pymongo import MongoClient
from io import StringIO
import io 
import time


# MongoDB setup
mongo_client = MongoClient("mongodb://localhost:27017/")
db = mongo_client["iot_database"]
collection = db["sensor_data"]

# URL endpoint for the streaming data
url = "https://www.floods.it/api/v1/monitoring/38561.csv"

def fetch_and_store_data():
    response = requests.get(url)
    if response.status_code == 200:
        # Read CSV data from the response
        data = response.content.decode('utf-8')
        df = pd.read_csv(io.StringIO(data))


        # Convert DataFrame to dictionary
        data_dict = df.to_dict("records")

        # Insert data into MongoDB
        if data_dict:
            collection.insert_many(data_dict)
            print(f"Inserted {len(data_dict)} records into MongoDB")
        else:
            print("No new data to insert")
    else:
        print(f"Failed to fetch data: {response.status_code}")

if __name__ == "__main__":
    while True:
        try:
            fetch_and_store_data()
        except Exception as e:
            print(f"An error occurred: {e}")
        # Wait for a while before fetching new data
        time.sleep(3599)  # Fetch data every 60 seconds


An error occurred: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 664b7d877319863b37f419dd, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>


KeyboardInterrupt: 